In [1]:
### Utility Functions
import pandas as pd
import sqlite3
from sqlite3 import Error
import datetime as dt
from collections import defaultdict

In [2]:
def create_connection(db_file, delete_db=False):
    import os
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)

    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn

In [3]:
def create_table(conn, create_table_sql, drop_table_name=None):
    
    if drop_table_name: # You can optionally pass drop_table_name to drop the table. 
        try:
            c = conn.cursor()
            c.execute("""DROP TABLE IF EXISTS %s""" % (drop_table_name))
        except Error as e:
            print(e)
    
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [4]:
def execute_sql_statement(sql_statement, conn):
    cur = conn.cursor()
    cur.execute(sql_statement)

    rows = cur.fetchall()

    return rows

In [5]:
def data_dictionary(data_filename):
    header = None
    with open(data_filename, 'r') as file:
        count = 0
        data_dict = defaultdict(list)
        for line in file:
            if not line.strip():
                continue
            if not header:
                header = line.strip().split('\t')
                continue
            line = line.strip().split('\t')
            for index, key in enumerate(header):
                data_dict[key].append(line[index])
    return data_dict

In [6]:
def step1_create_region_table(data_filename, normalized_database_filename):
    # Inputs: Name of the data and normalized database filename
    # Output: None
    
    ### BEGIN SOLUTION
    conn = create_connection(normalized_database_filename)
    cur = conn.cursor()
    
    ####################################################################################################
    ## Creating the table using "create_table" funtion
    
    create_table_sql = """CREATE TABLE IF NOT EXISTS[Region] (
                            [RegionID] INTEGER NOT NULL PRIMARY KEY,
                            [Region] TEXT NOT NULL);"""
    create_table(conn, create_table_sql, "Region")
    
    ####################################################################################################
    ## Looping Over the data to find the distinct regions
    
    data_dict = data_dictionary(data_filename)
    Regions = sorted(list(set(data_dict["Region"])))
    Regions = [(index+1, Region) for index, Region in enumerate(Regions)]
    
    ####################################################################################################
    ##Populating the Region table with the distinct regions.
    
    with conn:
        cur.executemany("INSERT INTO Region (RegionID, Region) VALUES (?, ?)", Regions)
    ### END SOLUTION

In [7]:
def step2_create_region_to_regionid_dictionary(normalized_database_filename):
    
    
    ### BEGIN SOLUTION
    conn = create_connection(normalized_database_filename)
    sql_statement = """SELECT * FROM Region;"""
    rows = execute_sql_statement(sql_statement, conn)
    region_to_regionid_dict = {}
    for value, key in rows:
        region_to_regionid_dict[key] = value
    return region_to_regionid_dict
    ### END SOLUTION

In [8]:
def step3_create_country_table(data_filename, normalized_database_filename):
    # Inputs: Name of the data and normalized database filename
    # Output: None
    
    ### BEGIN SOLUTION
    
    ##  Note: You will be using region_to_regionid_dict in this step when inserting into the country table. 
    ##        You can access region_to_regionid_dict by calling the function in step2 inside step3.
    ### Tasks:
    ###       1). Loop over the data and find the distinct countries and region combo
    ###       2). Create the country table
    ###       3). Populate the country table with the distinct countries and regio combo. 
    ###            Make sure to sort the countries before inserting.
    
    
    ##  Connecting to the Normalized_Database_Filename
    
    conn = create_connection(normalized_database_filename)
    cur = conn.cursor()
    
    ####################################################################################################
    ## Creating the table using "create_table" funtion
    
    create_table_sql = """CREATE TABLE IF NOT EXISTS[Country] (
                            [CountryID] INTEGER NOT NULL PRIMARY KEY,
                            [Country] TEXT NOT NULL,
                            [RegionID] INTEGER NOT NULL,
                            FOREIGN KEY(RegionID) REFERENCES Region(RegionID));"""
    create_table(conn, create_table_sql, "Country")
    
    ####################################################################################################
    ## Loop over the data and find the distinct countries and region combo
    region_to_regionid_dict = step2_create_region_to_regionid_dictionary(normalized_database_filename)
    data_dict = data_dictionary(data_filename)
    Countries = data_dict["Country"]
    Regions = data_dict["Region"]
    Country_Region_dict = dict(zip(Countries, Regions))
    
    for Region, RegionID in region_to_regionid_dict.items():
        for Country, Country_Region in sorted(Country_Region_dict.items()):
            if  Region == Country_Region:
                Country_Region_dict[Country] = RegionID
    
    Countries = []
    for index, (key, value) in enumerate(sorted(Country_Region_dict.items(), key = lambda ele:ele[0])):
        Countries.append((index+1, key,value))
   
    
    ####################################################################################################
    ##Populating the Region table with the distinct regions.
    with conn:
        cur.executemany("INSERT INTO Country (CountryID, Country, RegionID) VALUES (?, ?, ?)", Countries)
         
    ### END SOLUTION

In [9]:
def step4_create_country_to_countryid_dictionary(normalized_database_filename):
    
    
    ### BEGIN SOLUTION
    conn = create_connection(normalized_database_filename)
    cur = conn.cursor()
    sql_statement = """SELECT * FROM Country;"""
    rows = execute_sql_statement(sql_statement, conn)
    
    country_to_countryid_dictionary ={}
    for CountryID, Country, RegionID  in rows:
        country_to_countryid_dictionary[Country]= CountryID
    
    return country_to_countryid_dictionary

    ### END SOLUTION

In [10]:
def step5_create_customer_table(data_filename, normalized_database_filename):

    ### BEGIN SOLUTION
    
    ## You will be country_to_countryid_dict in this step when inserting into the customer table.
    ## You can access country_to_countryid_dict by calling the function in step4 inside step5.
    ##.  Tasks:
    ##.       1) Loop over the data file and grab the name, address, city, country and region.
    ##.       2) Create the customer table
    ##.       3) Populate the customer table
    
    
    ##  Connecting to the Normalized_Database_Filename
    
    conn = create_connection(normalized_database_filename)
    cur = conn.cursor()
    
    ####################################################################################################
    ## Creating the customer table using "create_table" funtion
    
    create_table_sql = """CREATE TABLE IF NOT EXISTS[Customer] (
                            [CustomerID] INTEGER NOT NULL PRIMARY KEY,
                            [FirstName] TEXT NOT NULL,
                            [LastName] TEXT NOT NULL,
                            [Address] TEXT NOT NULL,
                            [City] TEXT NOT NULL,
                            [CountryID] INTEGER NOT NULL,
                            FOREIGN KEY(CountryID) REFERENCES Country(CountryID));"""
    create_table(conn, create_table_sql, "Customer")
    
    ####################################################################################################
    ## Looping over the data file and grab the name, address, city, country and region.
    
    data_dict = data_dictionary(data_filename)
    Names = data_dict['Name']
    Addresses = data_dict["Address"]
    Cities = data_dict["City"]
    Countries = data_dict["Country"]
    
    ####################################################################################################
    ## Populating the customer table with Customer details

    country_to_countryid_dict = step4_create_country_to_countryid_dictionary(normalized_database_filename)
    Country_IDS = []
    for Country in Countries:
        for key,value in country_to_countryid_dict.items():
            if key == Country:
                Country_IDS.append(value)
    
    Customers = []
    
    for (Name, Address, City, Country_ID) in zip(*[Names, Addresses, Cities, Country_IDS]):
        Customers.append((Name, Address, City, Country_ID))
        
    Customers = sorted(Customers, key = lambda ele : ele[0])
    
    Customers_data = []
    for index, Customer in enumerate(Customers):
        Name, Address, City, Country_ID = Customer[0], Customer[1], Customer[2], Customer[3]
        if len(Customer[0].split(" ")) == 2:
            FirstName, LastName = Name.split(" ")
        elif len(Customer[0].split(" ")) == 3:
            FirstName, LastName =Name.split(" ")[0], Name.split(" ")[1]+" "+Name.split(" ")[2]
        Customers_data.append((index+1, FirstName, LastName, Address, City, Country_ID))

    with conn:
        cur.executemany("INSERT INTO Customer (CustomerID, FirstName, LastName, Address, City, CountryID) VALUES (?, ?, ?, ?, ?, ?)", Customers_data)

    ### END SOLUTION

In [11]:
def step6_create_customer_to_customerid_dictionary(normalized_database_filename):
    
    
    ### BEGIN SOLUTION
    #The purpose of this step is to create a dictionary to look up the primary key lookup for a given name (FirstName LastName)
    ##.     Tasks:
    ##.         1) Write an SQL query to fetch all the rows from the Customer table
    ##.         2) Transform the row into a dictionary whose key is the name of the customer (FirstName LastName) and value is CustomerID
    
    conn = create_connection(normalized_database_filename)
    cur = conn.cursor()
    sql_statement = """SELECT * FROM Customer;"""
    rows = execute_sql_statement(sql_statement, conn)
    
    customer_to_customerid_dictionary = {}

    for Customer in rows:

        CustomerID, FirstName, LastName = Customer[0], Customer[1], Customer[2]

        CustomerName = FirstName +" "+ LastName

        customer_to_customerid_dictionary[CustomerName] = CustomerID

    return customer_to_customerid_dictionary

    ### END SOLUTION

In [12]:
def step7_create_productcategory_table(data_filename, normalized_database_filename):
    # Inputs: Name of the data and normalized database filename
    # Output: None

    
    ### BEGIN SOLUTION
    # Tasks:
    #.      1) Loop over the data and find the distinct product categories
    #.      2) Create the product category table
    #.      3) Populate the product category table with the distinct product categories. 
    #.         Make sure to sort the categories before inserting.
    
    ### BEGIN SOLUTION
    ##  Connecting to the Normalized_Database_Filename
    
    conn = create_connection(normalized_database_filename)
    cur = conn.cursor()
    
    ####################################################################################################
    ## Creating the customer table using "ProductCateogry_table" funtion
    
    
    create_table_sql = """CREATE TABLE IF NOT EXISTS [ProductCategory] (
                            [ProductCategoryID] INTEGER NOT NULL PRIMARY KEY,
                            [ProductCategory] TEXT NOT NULL,
                            [ProductCategoryDescription] TEXT NOT NULL
                            );"""
    create_table(conn, create_table_sql, "ProductCategory")
    
    ####################################################################################################
    #. Looping over the data and find the distinct product categories
    data_dict = data_dictionary(data_filename)
    
    ProductCategories = data_dict["ProductCategory"]
    ProductCategories = [ProductCategory.split(";") for ProductCategory in ProductCategories]
    
    
    ProductCategoryDescriptions = data_dict["ProductCategoryDescription"] 
    ProductCategoryDescriptions = [ProductCategoryDescription.split(";") for ProductCategoryDescription in ProductCategoryDescriptions]
    
    
    ProductCategory_dict = dict(zip(ProductCategories[0], ProductCategoryDescriptions[0]))
    ProductCategory_dict = dict(sorted(ProductCategory_dict.items(), key = lambda ele : ele[0]))
    
    ProductCategory_data = []

    for index, (ProductCategory, ProductCategoryDescription)in enumerate(ProductCategory_dict.items()):

        ProductCategory_data.append((index+1, ProductCategory, ProductCategoryDescription))
      
    
    ####################################################################################################
    ## Populating the customer table with Customer details(Customers_data) after sorting by name
    
    with conn:
        cur.executemany("INSERT INTO ProductCategory (ProductCategoryID, ProductCategory, ProductCategoryDescription) VALUES (?, ?, ?)", ProductCategory_data)
   
    ### END SOLUTION

In [13]:
def step7_create_productcategory_table(data_filename, normalized_database_filename):
    # Inputs: Name of the data and normalized database filename
    # Output: None

    
    ### BEGIN SOLUTION
    # Tasks:
    #.      1) Loop over the data and find the distinct product categories
    #.      2) Create the product category table
    #.      3) Populate the product category table with the distinct product categories. 
    #.         Make sure to sort the categories before inserting.
    
    ### BEGIN SOLUTION
    ##  Connecting to the Normalized_Database_Filename
    
    conn = create_connection(normalized_database_filename)
    cur = conn.cursor()
    
    ####################################################################################################
    ## Creating the customer table using "ProductCateogry_table" funtion
    
    
    create_table_sql = """CREATE TABLE IF NOT EXISTS [ProductCategory] (
                            [ProductCategoryID] INTEGER NOT NULL PRIMARY KEY,
                            [ProductCategory] TEXT NOT NULL,
                            [ProductCategoryDescription] TEXT NOT NULL
                            );"""
    create_table(conn, create_table_sql, "ProductCategory")
    
    ####################################################################################################
    #. Looping over the data and find the distinct product categories
    data_dict = data_dictionary(data_filename)
    
    ProductCategories = data_dict["ProductCategory"]
    ProductCategories = [ProductCategory.split(";") for ProductCategory in ProductCategories]
    
    
    ProductCategoryDescriptions = data_dict["ProductCategoryDescription"] 
    ProductCategoryDescriptions = [ProductCategoryDescription.split(";") for ProductCategoryDescription in ProductCategoryDescriptions]
    
    
    ProductCategory_dict = dict(zip(ProductCategories[0], ProductCategoryDescriptions[0]))
    ProductCategory_dict = dict(sorted(ProductCategory_dict.items(), key = lambda ele : ele[0]))
    
    ProductCategory_data = []

    for index, (ProductCategory, ProductCategoryDescription)in enumerate(ProductCategory_dict.items()):

        ProductCategory_data.append((index+1, ProductCategory, ProductCategoryDescription))
      
    
    ####################################################################################################
    ## Populating the customer table with Customer details(Customers_data) after sorting by name
    
    with conn:
        cur.executemany("INSERT INTO ProductCategory (ProductCategoryID, ProductCategory, ProductCategoryDescription) VALUES (?, ?, ?)", ProductCategory_data)
   
    ### END SOLUTION

In [14]:
def step8_create_productcategory_to_productcategoryid_dictionary(normalized_database_filename):
    
    
    ### BEGIN SOLUTION
    ## The purpose of this step is to create a dictionary to look up the primary key lookup for a given product category
    ## Tasks:
    ##.     1) Write an SQL query to fetch all the rows from the product category table
    ##.     2) Transform the row into a dictionary whose key is the product category and value is product category id
    
    conn = create_connection(normalized_database_filename)
    cur = conn.cursor()

    sql_statement = """SELECT * FROM ProductCategory;"""

    rows = execute_sql_statement(sql_statement, conn)
    
    productcategory_to_productcategoryid_dictionary ={}

    for ProductCategoryID, ProductCategory, ProductCategoryDescription in rows:

        productcategory_to_productcategoryid_dictionary[ProductCategory] = ProductCategoryID

    return productcategory_to_productcategoryid_dictionary

    ### END SOLUTION

In [15]:
def step9_create_product_table(data_filename, normalized_database_filename):
    # Inputs: Name of the data and normalized database filename
    # Output: None

    
    ### BEGIN SOLUTION
    
    # Note: You will be using productcategory_to_productcategoryid_dict in this step when inserting into the product table.
    #.  Tasks:
    ##.      1) Loop over the data and find the distinct products
    ##.      2) Create the product table
    ##.      3) Populate the product table with the distinct products. 
    ##.        Make sure to sort the products (by name) before inserting.
    
    
    ##  Connecting to the Normalized_Database_Filename
    
    conn = create_connection(normalized_database_filename)
    cur = conn.cursor()
    
    ####################################################################################################
    ## Creating the customer table using "ProductCateogry_table" funtion
    
    
    create_table_sql = """CREATE TABLE IF NOT EXISTS [Product] (
                            [ProductID] INTEGER NOT NULL PRIMARY KEY,
                            [ProductName] TEXT NOT NULL,
                            [ProductUnitPrice] REAL NOT NULL,
                            [ProductCategoryID] INTEGER NOT NULL,
                            FOREIGN KEY(ProductCategoryID) REFERENCES ProductCategory(ProductCategoryID)
                            );"""
    create_table(conn, create_table_sql, "Product")
    
    ####################################################################################################
    ## Looping over the data and finding the distinct products
    data_dict = data_dictionary(data_filename)

    ProductNames = data_dict["ProductName"]
    ProductNames = [ProductName.split(";") for ProductName in ProductNames]
    
    
    ProductUnitPrices = data_dict["ProductUnitPrice"]
    ProductUnitPrices = [[float(ele) for ele in ProductUnitPrices.split(";")] for ProductUnitPrices in ProductUnitPrices]
    
    
    ProductCategories = data_dict["ProductCategory"]
    ProductCategories = [ProductCategory.split(";") for ProductCategory in ProductCategories]
    
    
    ProductCategory_IDS = step8_create_productcategory_to_productcategoryid_dictionary(normalized_database_filename)
    
    ProductCategories_IDS = []

    for ProductCategory in ProductCategories[0]:

        for key, value in ProductCategory_IDS.items():

            if key == ProductCategory:

                ProductCategories_IDS.append(value)
                
    Product_data = []  

    for ProductName, ProductUnitPrice, ProductCategoryID in zip(*[ProductNames[0], ProductUnitPrices[0], ProductCategories_IDS]):

        Product_data.append((ProductName, ProductUnitPrice, ProductCategoryID))
    
    
    Product_sorted_data = sorted(list(set(sorted(Product_data, key = lambda ele : ele[0]))))
    
    Product_table_data = []

    for index, (ProductName, ProductUnitPrice, ProductCategoryID) in enumerate(Product_sorted_data):

        Product_table_data.append((index+1,ProductName, ProductUnitPrice, ProductCategoryID))

    ####################################################################################################
    ##  Populating the product table with the distinct products after sorting by Product Name
    
    with conn:
        cur.executemany("INSERT INTO Product (ProductID, ProductName, ProductUnitPrice, ProductCategoryID) VALUES (?, ?, ?, ?)", Product_table_data)
   
    ### END SOLUTION

In [16]:
def step10_create_product_to_productid_dictionary(normalized_database_filename):
    
    ### BEGIN SOLUTION
    ## The purpose of this step is to create a dictionary to look up the primary key lookup for a given product
    ##    Tasks:
    ##.       1) Write an SQL query to fetch all the rows from the product table
    ##.       2) Transform the row into a dictionary whose key is the product and value is product id
    conn = create_connection(normalized_database_filename)
    cur = conn.cursor()
    sql_statement = """SELECT * FROM Product;"""
    rows = execute_sql_statement(sql_statement, conn)
    
    product_to_productid_dictionary = {}
    for ProductID, ProductName, ProductUnitPrice, ProductCategoryID in rows:
        product_to_productid_dictionary[ProductName] = ProductID
    
    return product_to_productid_dictionary

    ### END SOLUTION

In [17]:
def step11_create_orderdetail_table(data_filename, normalized_database_filename):
    # Inputs: Name of the data and normalized database filename
    # Output: None

    ### BEGIN SOLUTION
    ## Note: You will be using product_to_productid_dict and customer_to_customerid_dict in this step when inserting into the order detail table.
    ##      Tasks:
    ##.         1) Loop over the data and figure out the orders
    ##.         2) Create the orderdetails table
    ##.         3) Populate the orderdetails table with the orders
    
    ##  Connecting to the Normalized_Database_Filename
    
    conn = create_connection(normalized_database_filename)
    cur = conn.cursor()
    
    ####################################################################################################
    ## Creating the customer table using "ProductCateogry_table" funtion
    
    create_table_sql = """CREATE TABLE IF NOT EXISTS [OrderDetail] (
                            [OrderID] INTEGER NOT NULL PRIMARY KEY,
                            [CustomerID] INTEGER NOT NULL,
                            [ProductID] INTEGER NOT NULL,
                            [OrderDate] INTEGER NOT NULL,
                            [QuantityOrdered] INTEGER NOT NULL,
                            FOREIGN KEY(CustomerID) REFERENCES Customer(CustomerID),
                            FOREIGN KEY(ProductID) REFERENCES Product(ProductID)
                            );"""
    create_table(conn, create_table_sql, "OrderDetail")
    
    ####################################################################################################
    ## Looping over the data and figuring out the orders
    
    data_dict = data_dictionary(data_filename)
    
    CustomerIDS = step6_create_customer_to_customerid_dictionary(normalized_database_filename)
    ProductIDS  = step10_create_product_to_productid_dictionary(normalized_database_filename)
    
    Names = data_dict["Name"]
    
    ProductNames = data_dict["ProductName"]
    ProductNames = [ProductName.split(";") for ProductName in ProductNames]
    
    QuantityOrderds = data_dict["QuantityOrderded"]
    QuantityOrderds = [[int(ele)for ele in QuantityOrderded.split(";")] for QuantityOrderded in QuantityOrderds]
    
    OrderDates = data_dict["OrderDate"]
    OrderDates = [[dt.datetime.strptime(element, '%Y%m%d').strftime('%Y-%m-%d') for element in OrderDate.split(";")] for OrderDate in OrderDates]
    
    OrderDetails_NPDQ = []
 
    for Name, ProductName, OrderedDate, QuantityOrdered in zip(*[Names, ProductNames, OrderDates, QuantityOrderds]):

        for Product, Date, Quantity in zip(*[ProductName, OrderedDate, QuantityOrdered]):

            OrderDetails_NPDQ.append((CustomerIDS[Name], ProductIDS[Product], Date, Quantity))
            
    OrderDetails_Data = []
    
    for index, (Customer_ID, Product_ID, Order_Date,Quantity_Ordered) in enumerate(OrderDetails_NPDQ):
        
        OrderDetails_Data.append((index+1, Customer_ID, Product_ID, Order_Date, Quantity_Ordered))
        
      
    ####################################################################################################
    ##. Populating the orderdetails table with the orders
    
    with conn:
        cur.executemany("INSERT INTO OrderDetail (OrderID, CustomerID, ProductID, OrderDate, QuantityOrdered) VALUES (?, ?, ?, ?, ?)", OrderDetails_Data)
    ### END SOLUTION

In [18]:
def ex1(conn, CustomerName):
    
    # Simply, you are fetching all the rows for a given CustomerName. 
    # Write an SQL statement that SELECTs From the OrderDetail table and joins with the Customer and Product table.
    # Pull out the following columns. 
    # Name -- concatenation of FirstName and LastName
    # ProductName
    # OrderDate
    # ProductUnitPrice
    # QuantityOrdered
    # Total -- which is calculated from multiplying ProductUnitPrice with QuantityOrdered -- round to two decimal places
    # HINT: USE customer_to_customerid_dict to map customer name to customer id and then use where clause with CustomerID
    
    ### BEGIN SOLUTION
    Customer_IDS = step6_create_customer_to_customerid_dictionary("normalized.db")
    CustomerID = Customer_IDS[CustomerName]
    sql_statement = f"""SELECT Req_T.FirstName || ' ' || Req_T.LastName AS Name, Req_T.ProductName, Req_T.OrderDate, Req_T.ProductUnitPrice, Req_T.QuantityOrdered, ROUND(Req_T.ProductUnitPrice * Req_T.QuantityOrdered, 2) AS Total
                        FROM
                            (SELECT *
                                FROM OrderDetail OD 
                                    INNER JOIN Customer C
                                    ON OD.CustomerID = C.CustomerID
                                    INNER JOIN Product P
                                    ON P.ProductID =  OD.ProductID
                                    WHERE C.CustomerID= {CustomerID}) Req_T;
                    
    """
    ### END SOLUTION
    df = pd.read_sql_query(sql_statement, conn)
    return sql_statement

In [21]:
data_filename = 'data.csv'
normalized_database_filename = 'normalized.db'
conn =create_connection(normalized_database_filename, delete_db=True)

In [22]:
step1_create_region_table(data_filename, normalized_database_filename)

In [24]:
step3_create_country_table(data_filename, normalized_database_filename)

In [26]:
step5_create_customer_table(data_filename, normalized_database_filename)

In [28]:
step7_create_productcategory_table(data_filename, normalized_database_filename)

In [29]:
step9_create_product_table(data_filename, normalized_database_filename)

In [30]:
step11_create_orderdetail_table(data_filename, normalized_database_filename)

In [33]:
sql_statement = """SELECT Req_T.FirstName || ' ' || Req_T.LastName AS Name, Req_T.ProductName, Req_T.OrderDate, Req_T.ProductUnitPrice, Req_T.QuantityOrdered, ROUND(Req_T.ProductUnitPrice * Req_T.QuantityOrdered, 2) AS Total
                        FROM
                            (SELECT *
                                FROM OrderDetail OD 
                                    INNER JOIN Customer C
                                    ON OD.CustomerID = C.CustomerID
                                    INNER JOIN Product P
                                    ON P.ProductID =  OD.ProductID
                                    WHERE C.CustomerID= 12) Req_T
                                    LIMIT 10;"""
execute_sql_statement(sql_statement, conn)

[('Carine Schmitt', 'Pavlova', '2013-09-17', 17.45, 10, 174.5),
 ('Carine Schmitt', 'Tarte au sucre', '2013-09-17', 49.3, 10, 493.0),
 ('Carine Schmitt',
  'Louisiana Fiery Hot Pepper Sauce',
  '2013-09-17',
  21.05,
  12,
  252.6),
 ('Carine Schmitt', 'Manjimup Dried Apples', '2014-01-29', 53.0, 3, 159.0),
 ('Carine Schmitt', 'Lakkalikoori', '2014-01-29', 18.0, 20, 360.0),
 ('Carine Schmitt',
  'Thuringer Rostbratwurst',
  '2014-03-24',
  123.79,
  14,
  1733.06),
 ('Carine Schmitt', "Gustaf's Knackebrod", '2012-10-14', 21.0, 39, 819.0),
 ('Carine Schmitt', 'Gula Malacca', '2012-10-14', 19.45, 40, 778.0),
 ('Carine Schmitt', 'Tunnbrod', '2012-10-14', 9.0, 21, 189.0),
 ('Carine Schmitt', 'Ikura', '2012-10-14', 31.0, 12, 372.0)]